# Setup

## Imports

In [ ]:
import numpy as np
import os
import tqdm

In [ ]:
import yt
import unyt
import trident
import trident.firefly_generator as trident_firefly

In [ ]:
import firefly

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import palettable

In [ ]:
import verdict
import trove
import helpers

In [ ]:
from importlib import reload

## Parameters

In [ ]:
ions = [
    'H I',
    # 'O I',
    # 'C II',
    # 'C III',
    'C IV',
    # 'N II',
    # 'N III',
    # 'Si II',
    # 'Si III',
    # 'Si IV',
    # 'N V',
    'O VI',
    'Mg II'
]
field_limits = {
    'logHIdensity': [ -6, 3 ],
    'logCIVdensity': [ -15, -8 ],
    'logOVIdensity': [ -15, -8 ],
    'logMgIIdensity': [ -8, -1 ],
}
starting_colormap = 'logCIVdensity'
    
fields_to_include = [ 'temperature', 'density', 'metallicity' ]
fields_units = [ 'log(K)', 'log(mp/cm**3)', 'Zsun' ]
snr = 30

In [ ]:
pm = trove.link_params_to_config(
    helpers.CONFIG,
    script_id = 'nb.3',
    variation = 'm12i_md',
)

In [ ]:
qual_colors = palettable.cartocolors.qualitative.Vivid_10.mpl_colors

In [ ]:
sim = pm['variation']

## Load Halo Data

In [ ]:
halo_catalog_fn = 'halo_{}.hdf5'.format( pm['snum'] )
halo_catalog_fp = os.path.join( pm['rockstar_data_dir'], halo_catalog_fn )

In [ ]:
halo_data = verdict.Dict.from_hdf5( halo_catalog_fp )
index = halo_data['mass'].argmax()
center_ckpc = halo_data['position'][index]
center = center_ckpc / ( 1. + halo_data['snapshot:redshift'] )

## Load Simulation Data

In [ ]:
yt_sim_fp = os.path.join( pm['sim_data_dir'], 'snapdir_{:03d}'.format( pm['snum'] ) )
ds = yt.load( yt_sim_fp )

In [ ]:
kpc = ds.quan( 1, 'kpc' )

## Load Processed Data

In [ ]:
data_fp = os.path.join( pm['processed_data_dir'], 'data.h5' )
data = verdict.Dict.from_hdf5( data_fp, create_nonexistent=True )

# Make Firefly Visualization

## Simulation Data

In [ ]:
# Add ion fields
trident.ion_balance.add_ion_fields( ds, ions )

In [ ]:
# Ion units, names, and internal names
ldb = trident.line_database.LineDatabase( 'lines.txt' )
number_densities_to_include = [ ( 'gas', '{}_p{}_number_density'.format( atom, lev-1 ) ) for atom, lev in ldb.parse_subset_to_ions( ions ) ]
number_density_units = [ 'log(cm**-3)', ] * len( number_densities_to_include )
number_density_names = [ 'log{}density'.format( _.replace( ' ', '' ) ) for _ in ions ]

In [ ]:
# Setup a subvolume for easier management
sun_position = data[sim]['end'] * kpc
sp = ds.sphere( sun_position, (2. * halo_data['radius'][index], "kpc") )

In [ ]:
# Ensure we can output
os.makedirs( pm['firefly_dir'], exist_ok=True )

In [ ]:
ff_reader = sp.create_firefly_object(
    datadir = pm['firefly_dir'],
    ptypes = [ 'PartType0', ],
    fields_to_include = fields_to_include + number_densities_to_include,
    fields_units = fields_units + number_density_units,
    field_names = fields_to_include + number_density_names,
)

In [ ]:
# Decide on limits

%matplotlib inline

fig = plt.figure(figsize=( 5, 5 * len( ions ) ))
ax_dict = fig.subplot_mosaic(
    [ [ _, ] for _ in ions ],
)

for i, num_den in enumerate( number_densities_to_include ):
    ax = ax_dict[ions[i]]
    
    num_den_arr = sp[num_den].value
    
    xmin = np.nanmin( num_den_arr[num_den_arr>1e-16], )
    xmax = np.nanmax( num_den_arr, )
    
    ax.hist(
        num_den_arr,
        bins = np.logspace( np.log10( xmin ), np.log10( xmax ), 16 ),
        # log = True,
        color = 'k',
    )
    
    field_name = number_density_names[i]
    for ion_lim in field_limits[field_name]:
        ax.axvline(
            10.**np.array(ion_lim),
        )
    
    ax.set_xscale( 'log' )
    ax.set_xlim( xmin, xmax )

    ax.set_ylabel( ions[i] )


## Sightlines

In [ ]:
for i, start in enumerate( data[sim]['start'] ):
    
    print( 'Generating particle group for sightline {:03d}'.format( i ) )

    ray_fp = os.path.join( pm['data_dir'], 'sightlines', 'ray_{:03d}.h5'.format( i ) )
    ray = yt.load( ray_fp )

    ff_gen = trident_firefly.FireflyGenerator()
    ray_pg = ff_gen.create_particle_group(
        ray,
        UIname = 'data{:03d}'.format( i ),
        center = sp.center,
        coordinate_units = sp.center.units,
        lines = ions,
    )
    guideline_pg = ff_gen.create_particle_group_guideline(
        ray,
        UIname = 'LOS{:03d}'.format( i ),
        center = sp.center,
        coordinate_units = sp.center.units,
    )

    ff_reader.addParticleGroup( ray_pg )
    ff_reader.addParticleGroup( guideline_pg )

    # # Turn blending off to make the guidelines more visible
    # ff_reader.settings['blendingMode'][guideline_pg.UIname] = 'none'
    # ff_reader.settings['depthTest'][guideline_pg.UIname] = True

## Finish Up

In [ ]:
# Change limits
for field_name, limit in tqdm.tqdm( field_limits.items() ):
    for setting in [ 'colormapLims', 'filterLims' ]:
        settings_entry = ff_reader.settings[setting]
        for ptype, setting_values in settings_entry.items():
            if field_name in setting_values:
                settings_entry[ptype][field_name] = np.array( limit )

In [ ]:
# Turn colormaps on
if starting_colormap is not None:
    for pg_ in ff_reader.particleGroups:
        if starting_colormap in pg_.field_names:
            ff_reader.settings['colormapVariable'][pg_.UIname] = int( np.where( pg_.field_names == starting_colormap )[0] )
            ff_reader.settings['showColormap'][pg_.UIname] = True

## Output to JSON

In [ ]:
ff_reader.writeToDisk(loud=True,symlink=True,file_extension='.json')

## Run the Viz Itself

In [ ]:
from firefly.server import spawnFireflyServer, quitAllFireflyServers

In [ ]:
port = 5500

# Spawn a server to host the visualization
process = spawnFireflyServer(port=port)

In [ ]:
ff_reader.sendDataViaFlask(port)

In [ ]:
quitAllFireflyServers()